In [2]:
from pyspark.sql import SparkSession

# Create SparkSession (this automatically creates SparkContext internally)
spark = SparkSession.builder.master("local").appName("demo application").getOrCreate()

# You can access the SparkContext via the SparkSession object
sc = spark.sparkContext

# Print to check SparkContext creation
print("SparkContext created successfully")

# Parallelize the data using SparkContext (via SparkSession)
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)

# Collect and print the result
print(distData.collect())  # Should print [1, 2, 3, 4, 5]


SparkContext created successfully
[1, 2, 3, 4, 5]


# to start
First we hav e to create a spark sessionand we have to give it an appname and a master
the master will tell spark where the spark job needs to run, here it will be local because we are running it on our local machine.
Typically you would put an url for a spark cluster/mesos cluster or YARN if you were using that.


In [3]:
spark = SparkSession.builder.master("local").appName("demo application").getOrCreate()

create a dataframe to interact with (structured data in our case)
we read it from a csv but if it is an array you can use the parelilize method to create a dataframe from an array.
we need to also pass a schema when creating a dataframe (column names) our csv already has the headers so it will do it automatically also datatypes will be asssigned automatically

the \ are used so that cwe can make the code more readable since laot of .methods are being chained behind eachother.

we have to specify the path and then load the data into a dataframe.


In [4]:
data = spark.read.format('csv').option('interSchema', 'True').\
    option('header', 'True').\
        option('path', './data/operations_management.csv').\
            load()

In [5]:
data.printSchema()

root
 |-- description: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- level: string (nullable = true)
 |-- size: string (nullable = true)
 |-- line_code: string (nullable = true)
 |-- value: string (nullable = true)



as you can see it has created the dataframe with the correct datatypes for every column.


# TRANSFORMING DATA
like in pandas, you do not change the original data in the dataframe.

here we will select two columns, industry and value because in this example we are interested in that.

we filter so that we only have the industries that are worth over 1000 and then we sort them in descending order to get the most valuable ones first in this dataframe.

In [6]:
# create ac opy and apply transformation logic  to the dataframe
data2 = data.select("industry", "value").\
    filter(data["value"] > 10000).\
    orderBy(data["value"].desc())




In [7]:
data2.show()

+--------+-----+
|industry|value|
+--------+-----+
|   total|41091|
|   total|40431|
|   total|33984|
|   total|33750|
|   total|32652|
|   total|31134|
|   total|30624|
|   total|30543|
|   total|30396|
|   total|30099|
|   total|29994|
|   total|29775|
|   total|28473|
|   total|27846|
|   total|26010|
|   total|25977|
|   total|25887|
|   total|25500|
|   total|25434|
|   total|25221|
+--------+-----+
only showing top 20 rows



In [43]:
data2 = data.select("industry", "value").\
    filter((data["value"] > 200) & (data["industry"] != "total")).\
    orderBy(data["value"].desc())


In [44]:
data2.show()

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|      Other services|  999|
|Accommodation & f...|  999|
|        Construction|  999|
|Administrative & ...|  999|
|Administrative & ...|  999|
|     Wholesale trade|  999|
|     Wholesale trade|  996|
|     Wholesale trade|  996|
|Other wholesale t...|  996|
|       Manufacturing|  996|
|         Agriculture|  996|
|Professional, sci...|  996|
|Other professiona...|  996|
|Accommodation & f...|  993|
|Other wholesale t...|  993|
|      Other services|  993|
|Accommodation & f...|  990|
|     Wholesale trade|  990|
|Health care & soc...|  987|
|Agriculture, fore...|  987|
+--------------------+-----+
only showing top 20 rows



In [45]:
data2.createOrReplaceTempView("data")

In [50]:
spark.sql("""
          SELECT industry, value 
          FROM data 
          WHERE industry != 'total' AND value > 2000
          ORDER BY value DESC
          """).show()

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|        Construction| 6030|
|        Construction| 5904|
|        Construction| 5229|
|Accommodation & f...| 5058|
|        Construction| 4965|
|        Construction| 4959|
|Accommodation & f...| 4950|
|        Construction| 4686|
|        Construction| 4668|
|        Construction| 4665|
|       Manufacturing| 4662|
|       Manufacturing| 4632|
|        Construction| 4575|
|        Construction| 4566|
|Professional, sci...| 4476|
|Professional, sci...| 4470|
|        Retail trade| 4434|
|        Retail trade| 4434|
|Accommodation & f...| 4251|
|Accommodation & f...| 4176|
+--------------------+-----+
only showing top 20 rows



In [56]:
data3 = data.select("industry", "value").\
    filter((data["value"] > 200) & (data["industry"] != "total")).\
    orderBy(data["value"].desc())

data3 = data.withColumn("value", data["value"].cast("int"))

data3.createOrReplaceTempView("data3")

spark.sql("""
          SELECT industry, value 
          FROM data 
          WHERE industry != 'total' AND value > 2000 
          ORDER BY value DESC
          """).show(5)



+--------------------+-----+
|            industry|value|
+--------------------+-----+
|        Construction| 6030|
|        Construction| 5904|
|        Construction| 5229|
|Accommodation & f...| 5058|
|        Construction| 4965|
+--------------------+-----+
only showing top 5 rows



In [ ]:
data4 = spark.read.format('csv').option('interSchema', 'True').\
    option('header', 'True').\
        option('path', './data/purchase.csv').\
            load()

data4.createOrReplaceTempView("data4")

spark.sql("""
          SELECT Last_Purchase_Amount, Age
          FROM data4
          WHERE last_purchase_amount > 200
          ORDER BY last_purchase_amount DESC
""").show(5)


+--------------------+---+
|Last_Purchase_Amount|Age|
+--------------------+---+
|         7519.376995| 64|
|         7313.279661| 64|
|         7301.583331| 64|
|         7288.216661| 62|
|         7287.730593| 62|
+--------------------+---+
only showing top 5 rows



In [71]:
data4.write.csv(path='./data/purchase2.csv', header=True)


Py4JJavaError: An error occurred while calling o882.csv.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOu

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 43.0 failed 1 times, most recent failure: Lost task 0.0 in stage 43.0 (TID 41) (DESKTOP-NPG1LE2 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNex

In [ ]:
spark.sql("""
          SELECT industry, value 
          FROM data 
          WHERE industry != 'total' AND value > 2000
          ORDER BY value DESC
          """).show()

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|        Construction| 6030|
|        Construction| 5904|
|        Construction| 5229|
|Accommodation & f...| 5058|
|        Construction| 4965|
|        Construction| 4959|
|Accommodation & f...| 4950|
|        Construction| 4686|
|        Construction| 4668|
|        Construction| 4665|
|       Manufacturing| 4662|
|       Manufacturing| 4632|
|        Construction| 4575|
|        Construction| 4566|
|Professional, sci...| 4476|
|Professional, sci...| 4470|
|        Retail trade| 4434|
|        Retail trade| 4434|
|Accommodation & f...| 4251|
|Accommodation & f...| 4176|
+--------------------+-----+
only showing top 20 rows

